# Fast Db Searcher
This Jupyter notebook aims to convert natural language queries into SQL statements, leveraging a large language model (LLM). The goal is to accomplish this using the `Llama 3.3 8B model` while minimizing the number of LLM calls, therefore reducing latency.

The notebok is divided in two main section, a `router section` and a `converter section`. The `router section` will decide which table(s) needs to be queried, while the `converter section` will actually perform the translation from natural language to sql statements.

![intro_schema](images/general_schema.png)

Only the converter section leverages LLM while the router section exploits semantic similarity to pick the proper tables in the database.

For sake of simplicity, we'll be working with a simple fake database made of 2 tables. A table regarding `countries` data and one regarding `cars` data. 
The router will decide which tables to be queried according the human query. If the natural language query is categorized under the "countries" topic, the "country" table will be queried, if the natural language query is categorized  under the "cars" topic, the "cars" table will be queried. In addition, if the natural language query is NOT categorized either under the "countries" topic nor under the "cars" topic, no tables will be queried. Finally the converter will do the job of converting the natural language query in SQL query based on the information of the chosen table (table schema, table sample, ...).

## Router
As we mentioned earlier, the purpose of the router is to decide whether a human query refers to the to `countries` topic, `cars` topic or if it refers to no topic at all.

To accomplish this task, we leverage a library called `semantic router`. The `semantic router` library allows to assess whether a natural language query is related to a specific topic by comparing the query with some utterances. The utterances are specific verbal expressions used to define the semantic field that we want to enclose.


![SEMANTIC_ROUTER_SCHEMA](images/semantic_router_schema.png)

In simple words, the query is compared to each utterance by vector similarity. Therefore, a similarity score is computed between each utterance and query. Based on the similarity score, the router will decides if the query belongs to the cars topic, countries topic or none of them.

Now let's get to the code!

Install the necessary packages

In [1]:
!pip freeze > requirements.txt

Import the required packages

In [ ]:
from semantic_router import Route
import os
import json
from tqdm import tqdm
from semantic_router.encoders import OpenAIEncoder
from semantic_router.routers import SemanticRouter

Define the encoder to create embeddings for queries and utterances

In [3]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
encoder = OpenAIEncoder(score_threshold=0.16) # you can tune this parameter to steer the performance of your router

Define the utterances for countries data and cars data

In [17]:
CAR_UTTERANCES = [
    "car",
    "vehicle",
    "engine",
    "speed",
    "drive",
    "wheels",
    "fuel",
    "horsepower",
    "brake",
    "race",
    "ratings",
    "manufacture",
    "steering wheel",
    "dashboard",
    "acceleration",
    "sales",
    "production",
    "kilometer"
]

COUNTRIES_UTTERANCES = [
    "land",
    "nation",
    "country",
    "homeland",
    "flag",
    "borders",
    "culture",
    "history",
    "cuisine",
    "traditions",
    "rivers",
    "cities",
    "passport",
    "landscape",
    "mountains",
    "fauna", 
    "flora",
    "government",
    "capital",
    "citizen",
    "language",
    "economy",
    "currency",
    "anthem",
    "constitution"
    "population",
    "forest",
    "desert",
    "coastline",
    "military",
    "politics",
    "province",
]


Define routes based on the utterances

In [22]:
cars = Route(name="cars", utterances=CAR_UTTERANCES)
countries = Route(name="countries", utterances=COUNTRIES_UTTERANCES)
routes = [cars, countries]

Define the router based on the encoder and the routes

In [ ]:
rl = SemanticRouter(encoder=encoder, routes=routes, auto_sync="local")

Now it is all set! We can test the router on a bunch of queries, For this example we will use the queries in queries.json. This file contains 10 queries about the cars data, 10 queries about the countries data and 10 queries about unrelated topics. We will run the router on this 30 queries and create a report to evaluate the performance of the router. The report is saved under the name report.json

In [ ]:
# load queries
with open("queries.json", "r") as file:
    queries = json.load(file)

# build report
report = []

for query in tqdm(queries, desc="Processing questions"):
    route = rl(query["query"]).name

    # fill out report
    report.append(
        {
            "query": query["query"],
            "category": query["category"],
            "route": route,
        }
    )


# Save report
with open("report.json", "w") as file:
    file.write(json.dumps(report, indent=4))

Let's comment the results of the report. The 10 queries about cars were correctly categorized under the cars topic. On the other hand, for what concerns the countries queries, 9/10 were correctly categorized under the country topic. Finally, about the unrelated queries, only 7/10 were correctly categorized as unrelated.

Let's have a closer look at what the router misclassified to see if we can improve its performance. Consider the following query:

`{
    "query": "In which place is the horse the most common animal?",
    "category": "countries",
    "route": "cars"
},`

This query refers to country data but it is categorized under the cars topic by the router. Why? If I look up at my countries utterances, I see that I included the word "fauna" so I'm expecting the router to categorize this query properly. However, If I look also at the car utterances, I see the word "horsepower", so that's what led the router to misclassify the query. 

Consider this other query:

`{
    "query": "Explain the theory of relativity.",
    "category": "unrelated",
    "route": "cars"
},`

This query is clearly not related to "countries" and "cars" topics but it was erraneously categorized under the cars topic by the router. Why? It's probably due to the word "speed" appearing in the car utterances. The theory of relativity is strictly related to the speed of light, thus leading the router to misclassify the query. 

Is there a way to fix this? We can try to work on the utterances. Let's redefine the utterances in the following way:


In [21]:
CAR_UTTERANCES = [
    "fuel sales vehicle speed",
    "sales ratings engine horsepower",
    "brake steering wheel manufacture acceleration",
    "race brake vehicle engine",
    "fuel sales acceleration brake",
    "acceleration kilometer brake sales",
    "dashboard fuel wheels car",
    "vehicle manufacture drive horsepower",
    "ratings dashboard vehicle production",
    "race kilometer drive vehicle",
    "drive car engine speed",
    "production manufacture vehicle wheels",
    "fuel race horsepower speed",
    "ratings steering wheel race acceleration",
    "drive manufacture speed kilometer",
    "brake acceleration dashboard manufacture",
    "steering wheel vehicle manufacture kilometer",
    "manufacture car horsepower wheels",
    "race fuel speed brake",
    "car dashboard fuel sales",
    "manufacture production race speed",
    "brake manufacture acceleration drive",
    "steering wheel brake fuel ratings",
    "steering wheel drive dashboard production",
    "car speed sales race",
    "ratings steering wheel sales manufacture",
    "fuel drive acceleration brake",
    "drive ratings engine speed",
    "wheels drive vehicle acceleration",
    "dashboard sales manufacture acceleration",
]

COUNTRIES_UTTERANCES = [
    "flag province cities coastline",
    "land traditions fauna citizen",
    "fauna anthem homeland cuisine",
    "military nation flora borders",
    "flag military forest coastline",
    "mountains culture currency rivers",
    "cities desert capital politics",
    "flora culture government homeland",
    "anthem land flag culture",
    "military culture currency cuisine",
    "flora cities coastline anthem",
    "province flora citizen land",
    "nation country currency coastline",
    "province flora mountains desert",
    "language cities flag constitutionpopulation",
    "government military homeland flora",
    "politics flora mountains capital",
    "constitutionpopulation economy anthem currency",
    "country nation anthem traditions",
    "fauna coastline province culture",
    "flag currency fauna military",
    "economy culture politics cities",
    "province currency desert land",
    "flora traditions cuisine passport",
    "passport homeland anthem capital",
    "economy citizen capital cuisine",
    "forest anthem capital country",
    "nation country forest anthem",
    "citizen cuisine coastline history",
    "history language economy culture"
]


As shown above, we redefined the utterances for each category. Each utterance is a random combination of the 4 elements taken from the previous single utterances. In this way the concept of the speed will be more closely related to the cars topic by utterances such as "drive car engine speed". The same goes for the concept of horsepower: its relation with the car world will be further increased by utterances such as "fuel race horsepower speed". In simple words, we enforced the utterances to be closely related to their topics of interest (cars and countries).

Let's now redefine the routes, redefine the router and run the test once again. Now we can see the that each query was correctly categorized by the router!

## Converter
Now that we know on which table we have to work on, we can easily convert the natural language query into sql query by carefully prompting the large language model. The core idea is to build a prompt with all the necessary ingredients: natural language query, schema of table(s) of interest, and sample data of the table(s) of interest.

Import the required packages

In [25]:
from openai import OpenAI
from utils.helper import get_column_info, get_table_sample
import json
import sqlite3
import os


Define the test query and compute its route

In [28]:
test_query = "Who between Vortex X1 and Avalon S has the highest fuel consumption?"
query_route = rl(test_query).name
print(f"route: {query_route}")

2025-05-18 17:00:34 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


route: cars


Get the info of the table of interest according to the route

In [29]:
db_path = "data/data.db"
if query_route == "cars":
    table_name = "cars_data"
    column_info = get_column_info(db_path, "cars_data")
    table_sample = get_table_sample(db_path, "cars_data")

elif query_route == "countries":
    table_name = "countries_data"
    column_info = get_column_info(db_path, "countries_data")
    table_sample = get_table_sample(db_path, "countries_data")

Define the template of the prompt to be used

In [30]:
NL2SQL_TEMPLATE = """
You are a software engineer specialized in converting natural language query to SQL query. Follow closely the provided instructions. 

# Instructions
- Always look up closely to the 'database info' section before converting the natural language query into a SQL query
- Always follow the provided output format
- ONLY include the SQL query to the user in your response

# Database info
<column_info table_name="{table_name}">
Info about the columns of the table:
{column_info}
</column_info>

<table_sample table_name="{table_name}">
The first three rows of the table:
{table_sample}
</table_sample>

# Output format
Always provide the response according to the following output format:
{{"sql_query": <sql_query>}}

# Natural language query
{nl_query}
"""

Notice that the above template is scalable with respect to the number of tables involved since the "database info" section can be modified to account for multiple tables as long as we enclose them in the proper XML tags. For this case scenario we only have 1 table per topic, so it's okay.

Now let's define the prompt and the large language model. As we anticipated in the beginning of the notebook, the employed model is Llama-3.3-8b. 

In [31]:
# build prompt
prompt = NL2SQL_TEMPLATE.format(
    table_name=table_name,
    table_sample=table_sample,
    column_info=column_info,
    nl_query=test_query,
)

# define and invoke LLM
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)

completion = client.chat.completions.create(
  model="meta-llama/llama-3.3-8b-instruct:free",
  messages=[
    {
      "role": "user",
      "content": prompt
    }
  ]
)

response = completion.choices[0].message.content
print(response)

2025-05-18 17:00:47 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


{"sql_query": "SELECT car_name, fuel_consumption FROM cars_data WHERE car_name IN ('Vortex X1', 'Avalon S') ORDER BY fuel_consumption DESC LIMIT 1"}


Extract the SQL query from the response

In [32]:
response_dictionary = json.loads(response)
sql_query = response_dictionary["sql_query"]

Run the SQL in your database

In [33]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(sql_query)
result = cursor.fetchall()
print("SQL Query Result:")
for row in result:
    print(row)
conn.close()

SQL Query Result:
('Vortex X1', '7.5L/100km')


The SQL query yielded the expected result!